In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from collections import OrderedDict
from src.hdmm.workload import AllRange

In [2]:
pd.set_option('display.float_format', '{:0.2f}'.format)

In [3]:
def getBounds(df):
    """
    Returns [upper bound error, lower bound error]
    """
    return [df.abs_error.min(), df.abs_error.max()]

def getAverageError(df):
    """
    Returns average error across all queries
    """
    return df.abs_error.sum() / len(df)

def printBoundsAndAvgError(df):
    print(f'Average error is {getAverageError(df)}. Lower bound is {getBounds(df)[0]} and upper bound is {getBounds(df)[1]}')

In [4]:
def pmw2(workload, x, T, eps=0.01, k=0, show_messages=True, to_return='pd', analyst_labels = [],
       show_plot=False, show_failure_step=True):
    """
    Implement Private Multiplicative Weights Mechanism (PMW) on a workload of
    linear queries. New arguments to allow for optimizing the amount of
    privacy budget used in each step.
    
    to_return argument determines what the function will return. 
        - if 'pd', pmw() returns pandas df with test data for each 
        query (query, d_t_hat, updated, algo_ans, real_ans, abs_error, 
        rel_error). 
        - if 'update_count', pmw() returns the update count for the total
        amount of queries

    - W = workload of queries (M x k numpy array)
    - x = true database (M x 1 numpy array)
    - eps = privacy budget
    - k = number of update steps
    - T  = update threshold
    """ 
    
    # initialize constants
    m = x.size  # database len
    n = x.sum()
    eta = (math.log(m, np.e) ** (1 / 4)) / (math.sqrt(n))
    # Set k to be the desired number of update rounds
    if k==0: # essentially, if k hasn't been changed from its default value, use the length of the workload
        k = len(workload)  # num of queries
    delta = 1 / (n * math.log(n, np.e))
    x_norm = x / np.sum(x)
    # synthetic databases at time 0 (prior to any queries)
    y_t = np.ones(m) / m
    x_t = np.ones(m) / m

    # append to list of databases y_t and x_t
    x_list = [x_t]
    
    update_list = []
    update_count = 0
    pmw_answers = []
    update_times = []
    d_t_hat_list = []
    
    failure_mode = False # if reaches failure, failure mode because true and only runs lazy rounds
    
    def lazy_round():
        update_list.append('no')
        pmw_answers.append(np.dot(query, x_list[time]))
        x_list.append(x_list[time].round(3))
    
    #inititate first instance of SVT with half the budget and k updates
    #will be reset in the main loop
    SVTtrigger = False 
    SVTepsilon1 =((eps/2)/2)
    SVTepsilon2 = ((eps/2)/2)
    rho = np.random.laplace(loc=0, scale=(1/SVTepsilon1), size=1)[0]
    # iterate through time 
    for time, query in enumerate(workload):
        # Do one round of sparce vector technique 
        
        # compute noisy answer by adding Laplacian noise
        a_t = np.random.laplace(loc=0, scale=(2*k/SVTepsilon2), size=1)[0]
        a_t_hat = (np.dot(query, x_norm)*n ) + a_t

        # difference between noisy and maintained histogram answer
        d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
        
        # lazy round: use maintained histogram to answer the query
        if (abs(d_t_hat) <= T + rho) or failure_mode:
            d_t_hat_list.append(d_t_hat)
            lazy_round()
            continue

        # update round: update histogram and return noisy answer
        else:
            update_times.append(time)
            #make a new noisy query answer using some of the leftover budget
            a_t = np.random.laplace(loc=0, scale=(2*k/eps), size=1)[0]
            a_t_hat = (np.dot(query, x_norm)*n ) + a_t
            d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
            d_t_hat_list.append(d_t_hat)
            
            # step a
            if d_t_hat < 0:
                r_t = query
            else:
                r_t = np.ones(m) - query
            for i, v in enumerate(y_t):
                #y_t[i] = x_list[time][i] * math.exp(-eta * r_t[i])
                y_t[i] = x_list[time][i] * math.exp((d_t_hat/(2*n)) * query[i])
            
            #print(d_t_hat)
            #print(query)
            #print(x_list[time])
            # step b
            x_t = y_t / np.sum(y_t)
            update_count = update_list.count('yes')
            
            if update_count >= k: # threshold for num updates is reached, enter failure_mode
                failure_mode = True
                if show_failure_step:
                    print(f'Failure mode reached at query number {time}: {query}')
                lazy_round()
            else: # threshold for num updates is not reached yet
                x_list.append(x_t.round(3))
                update_list.append('yes') # increment number of updates counter
                pmw_answers.append(a_t_hat / np.sum(x))
                
    update_count = update_list.count('yes')      

    # calculate error
    real_ans = np.matmul(workload, x_norm)
    abs_error = np.abs(pmw_answers - real_ans)
    rel_error = np.abs(abs_error / np.where(real_ans == 0, 0.000001,
                                                real_ans))
    
    def print_outputs():
        np.set_printoptions(suppress=True)
        """Print inputes/outputs to analyze each query"""
        print(f'Original database: {x}\n')
        print(f'Normalized database: {x_norm}\n')
        print(f'Updated Database = {x_t}\n')
        print(f'Update Count = {update_count}\n')
        print(f'{T=}\n')
        print(f'Error Scale Query Answer= {2*((2*k/eps)**2)}\n')
        print(f'Error Scale SVT= {2*((2*k/SVTepsilon2)**2)}\n')
        print(f'Update Parameter Scale = {eta}\n')
        print(f'{delta=}\n')
    
    if show_messages:
        print_outputs()
        
    if show_plot: 
        plt.xticks(range(0, len(workload), 5))
        plt.title('Error across queries:')
        #rel_line, = plt.plot(rel_error, label='Relative Error')
        abs_line, = plt.plot(abs_error, label='Absolute Error')
        for xc in update_times:
            plt.axvline(x=xc, color='red', label='Update Times', linestyle='dashed')
        plt.legend(handles=[abs_line]) #,rel_line])
        
    if to_return == "pd":
        x_list.pop(0).tolist() # remove the first synthetic database to keep length of lists consistent-x_list[t] represents the synthetic database at the end of time t
        
        
        d = {
            'algo_ans': pmw_answers,
            'real_ans': real_ans.tolist(),
            'queries': workload.tolist(), 
            'updated': update_list,
            'abs_error': abs_error,               
#            'rel_error': rel_error,
            'synthetic database': x_list,
            'analyst': analyst_labels,
            'd_t_hat': d_t_hat_list, 

             }
        test_data = pd.DataFrame(data=d)
        test_data = test_data.round(3)
        return test_data
    
    if to_return == "error":
        #unique_analysts = list(OrderedDict.fromkeys(analyst_labels))
        
        d = {'analyst': analyst_labels,
             'abs_error': abs_error,               
             'rel_error': rel_error,}
        data = pd.DataFrame(data=d)
        data = data.round(3)
        
        analyst_error = {}
        for analyst in list(set(analyst_labels)):
            analyst_error[analyst] = data[data.analyst==analyst]['abs_error'].sum()
        return analyst_error

        

### Initializing workloads and databases

In [5]:
x_small = np.array([20, 160, 20, 20, 20, 160, 20, 20])
normalized = x_small / x_small.sum()
m = x_small.size  # database len
n = x_small.sum()
#print(f'the threshold for failure is {n * math.log(m, np.e) ** (1 / 2)}')

random_array = np.random.randint(2, size=(500,4))
zero_array = np.zeros((500,4))
alice = np.hstack((random_array, zero_array))
bob = np.hstack((zero_array, random_array))

We've shown that Alice can use up all the privacy budget. 

Next, let's take some of the basic adaptations of pmw and see if we can get the same effects to happen or a similar effect were people get similar results indpenedently

### To Dos: 

Instead of having a communal pool of update steps we can do, split the amount of update steps evenly across all analysts. If your update steps are not a multiple of the analysts, you can change the number of update steps.  

Look for the same kind of violation where either Bob or Alice will have more overall error in the joint case as opposed to the independent case.

In the independent state, you can use the existing pmw algorihtm.

Suggestion: When you split it to their individual settings, give them the same amount of update steps that they would've gotten in the group. 

Individually - 1 ep, 5 k. together - 2 ep, 10k. 

### Experiments: 
Try original pmw, adapted w equal update steps pmw, individual: 
1. [done] A and B query disjoint sections
2. [done] A queries the entire dataset except for last index. B queries entire dataset, last index inclusive. 
3. [done] A asks singleton, B asks all range queries
4. [if time] Is total error the right metric? Context: You might run into a case where you get flat error across all of your queries. You may get a lot worse error on one specific query, but the overall error is better. Now Alice and Bob care about the same data. Alice eats all the budget (entire database except for last). Bob cares about entire database, but his queries about the last index has higher error. 
    
    
Scenarios to look for: 
1. Regular pmw performs poorly, but adapted pmw works well. (probably in disjoint setting)
2. Scenarios where regular AND adapted pmw perform poorly

## Scenario 1: Disjoint

In [6]:
# initialize
s1random_array = np.random.randint(2, size=(25,4))
s1zero_array = np.zeros((25,4))
s1alice_q = np.hstack((s1random_array, s1zero_array))
s1bob_q = np.hstack((s1zero_array, s1random_array))

s1combined_list = []
s1bobfirst_list = []
s1individual_list = []

for i in range(1000):
    # combined
    s1combined = pmw2(workload=np.vstack((s1alice_q, s1bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1combined_list.append(s1combined)
    
    s1bobfirst = pmw2(workload=np.vstack((s1bob_q, s1alice_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Bob'] * 25 + ['Alice'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1bobfirst_list.append(s1bobfirst)
    
    # individual
    s1a = pmw2(workload=s1alice_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Alice'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1b = pmw2(workload=s1bob_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Bob'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1individual = {**s1a, **s1b}
    s1individual_list.append(s1individual)
    
# find mean over multiple trials
s1combined_average = dict(pd.DataFrame(s1combined_list).mean())
s1bobfirst_average = dict(pd.DataFrame(s1bobfirst_list).mean())
s1individual_average = dict(pd.DataFrame(s1individual_list).mean())

d = {'alice_first': s1combined_average, 'individual': s1individual_average, 'bob_first': s1bobfirst_average}
df = pd.DataFrame(data=d).sort_index()
display(df)

,alice_first,individual,bob_first
Alice,1.72,2.60,3.41
Bob,3.41,2.60,1.72


Bob always receives more error in the combined setting than in the individual setting.

Questions: 

1. Why does Bob always receives more error than Alice in the individual setting if their queries are not related? I would expect instead that their individual error would be very similar. **Bob's spike is larger**
    
2. Alice receives substantially more error in the individual setting than the combined setting. Is this supposed to happen? Although she receives less update steps in the individual setting, shouldn't the smaller epsilon value in the inidivual case counteract this effect? **Alice gets more update steps in the combined setting (eats Bob's)**


In [7]:
printBoundsAndAvgError(pmw2(workload=np.vstack((s1alice_q, s1bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='pd', show_plot=False, show_messages=False, show_failure_step=False))

Average error is 0.10342000000000001. Lower bound is 0.0 and upper bound is 0.245


We measure average error becauase we want to think about infinite query sequences - the types of sequences that PMW are very good at. Average error is better for longer query sequences. PMW is designed to make guarantees on average error. The error will be less than $\alpha$ with some probability of failure $\beta$.

# Scenario 2: Last Index

In [8]:
s2combined_list = []
s2individual_list = []
s2bobfirst_list = []

for i in range(1000):
    # combined
    s2alice_q = np.hstack((np.random.randint(2, size=(25,7)), np.zeros((25,1))))
    s2bob_q = np.random.randint(2, size=(25,8))

    s2combined = pmw2(workload=np.vstack((s2alice_q, s2bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s2combined_list.append(s2combined)

    
    s2bobfirst = pmw2(workload=np.vstack((s2bob_q, s2alice_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels= ['Bob'] * 25 + ['Alice'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s2bobfirst_list.append(s1bobfirst)
    
    # individual
    s2a = pmw2(workload=s2alice_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Alice'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s2b = pmw2(workload=s2bob_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Bob'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s2individual = {**s2a, **s2b}
    s2individual_list.append(s2individual)

# find mean over multiple trials
s2combined_average = dict(pd.DataFrame(s2combined_list).mean())
s2individual_average = dict(pd.DataFrame(s2individual_list).mean())
s2bobfirst_average = dict(pd.DataFrame(s1bobfirst_list).mean())

d = {'alice_first': s2combined_average, 'individual': s2individual_average, 'bob_first': s2bobfirst_average}
df = pd.DataFrame(data=d).sort_index()
display(df)

,alice_first,individual,bob_first
Alice,1.90,2.82,3.41
Bob,3.19,2.84,1.72


all indices except last -> all indices: 1.29 difference
all indices -> all indices except last: 1.69 difference

Alice is doing better than Bob in the individual and combined setting. 

When Bob goes first, he suffers less error than alice did when she went first. 

**Alice's error** in `bob_first` > **Bob's error** in `alice_first`. This makes the empirical point that if the second person queries less from the dataset than the first person, they face more error than going second if you query more from the dataset than the first person. This doesn't make sense to me. Shouldn't the second person that explores more of the dataset that has been previously unexplored experience more error than the second person in the other scenario that explores less of the dataset that had already been previous explored?

In [9]:
printBoundsAndAvgError(pmw2(workload=np.vstack((s2alice_q, s2bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='pd', show_plot=False, show_messages=False, show_failure_step=False))

Average error is 0.09866000000000001. Lower bound is 0.0 and upper bound is 0.329


# Scenario 3: Incompatible Queries

In [10]:
s3combined_list = []
s3individual_list = []
s3bobfirst_list = []

for i in range(1000):
    lst = [[0] * 8 for i in range(36)]
    for i in range(len(lst)):
        lst[i][np.random.randint(0, 8)] = 1
    s3bob_q = np.array(lst)
    s3alice_q = AllRange(8).dense_matrix()

    # combined
    s3combined = pmw2(workload=np.vstack((s3alice_q, s3bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 36 + ['Bob'] * 36, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s3combined_list.append(s3combined)
    
    
    s3bobfirst = pmw2(workload=np.vstack((s3bob_q, s3alice_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Bob'] * 36 + ['Alice'] * 36, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s3bobfirst_list.append(s3bobfirst)
    
    # individual
    s3a = pmw2(workload=s3alice_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Alice'] * 36, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s3b = pmw2(workload=s3bob_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Bob'] * 36, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s3individual = {**s3a, **s3b}
    s3individual_list.append(s3individual)
    
# find mean over multiple trials
s3combined_average = dict(pd.DataFrame(s3combined_list).mean())
s3individual_average = dict(pd.DataFrame(s3individual_list).mean())
s3bobfirst_average = dict(pd.DataFrame(s3bobfirst_list).mean())

d = {'alice_first': s3combined_average, 'individual': s3individual_average, 'bob_first': s3bobfirst_average}
df = pd.DataFrame(data=d).sort_index()
display(df)

,alice_first,individual,bob_first
Alice,2.68,3.31,3.48
Bob,3.91,3.56,2.87


Alice = All Range

Bob = Singleton

In All Range -> Singleton case, 1.22 more error for second person

In Singleton -> All Range case, 0.62 more error for second person

This makes sense to me because the All Range workload includes the Singleton queries, so the synthetic database had already been updated to respond to those types of queries.

In [11]:
printBoundsAndAvgError(pmw2(workload=np.vstack((s3alice_q, s3bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 36 + ['Bob'] * 36, 
                                 to_return='pd', show_plot=False, show_messages=False, show_failure_step=False))

Average error is 0.08705555555555555. Lower bound is 0.0 and upper bound is 0.238


# experiment 4: exact same workload

In [12]:
s4combined_list = []
s4individual_list = []
s4bobfirst_list = []

for i in range(1000):
    s4alice_q = np.random.randint(2, size=(25,8))
    s4bob_q = s4alice_q

    # combined
    s4combined = pmw2(workload=np.vstack((s4alice_q, s4bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s4combined_list.append(s4combined)
    
    s4bobfirst = pmw2(workload=np.vstack((s4bob_q, s4alice_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Bob'] * 25 + ['Alice'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s4bobfirst_list.append(s4bobfirst)
    
    # individual
    s4a = pmw2(workload=s4alice_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Alice'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s4b = pmw2(workload=s4bob_q, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Bob'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s4individual = {**s4a, **s4b}
    s4individual_list.append(s4individual)
    
# find mean over multiple trials
s4combined_average = dict(pd.DataFrame(s4combined_list).mean())
s4individual_average = dict(pd.DataFrame(s4individual_list).mean())
s4bobfirst_average = dict(pd.DataFrame(s4bobfirst_list).mean())

d = {'alice_first': s4combined_average, 'individual': s4individual_average, 'bob_first': s4bobfirst_average}
df = pd.DataFrame(data=d).sort_index()
display(df)

,alice_first,individual,bob_first
Alice,1.92,2.83,3.06
Bob,3.06,2.83,1.90


Makes sense to me. We are just switching the orders in which Alice and Bob are querying, and they have the exact same queries. 

In [13]:
printBoundsAndAvgError(pmw2(workload=np.vstack((s4alice_q, s4bob_q)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='pd', show_plot=False, show_messages=False, show_failure_step=False))

Average error is 0.08650000000000002. Lower bound is 0.002 and upper bound is 0.28


# To Do's (10/12)
1. [DONE] Do 1000 Trials for scenario 2 and 3
2. [DONE] Explaining the scenarios (why we chose these scenarios)
- Output the 3 tables. Practice explaining the tables to someone who doesn't know what's going on. 
- The general idea is that we want to show that the problem exist. These experiments show that standard online answering algorithms still have problems. 
    - experiment 1: disjoint, most adversarial setting. 
    - experiment 2: if we make a less extreme setting, alice only cares about most of the dataset, we still have this problem 
    - experiment 3: even when we use the entire database and use different types of database, we still run into this problem. Bob's workload is even embedded in Alice's workload!!
    - experiment 4: give them the exact same things, and bob is stil worse off. 
        - opposite of the free-rider problem - some folks can freely benefit from the public road without paying for it. in this case, if Bob had better error than Alice, then it would be free-rider problem. Not only are update steps better for making ur synthetic database better, update steps are better than any synthetic answer at all
    
    
    
- the multiplicative update step (non-private) = no regret learning - suppose there are true weights you give to weather experts who are guessing the weather - everytime you make an update step, the relative entropy between the weights you have and the true weights should go down. This is an average case guarantee. the problem with fairness topics is that average means nothing. 
- after tuesday (next friday) - implement PMW where each analyst is given some fraction of the total update stpes and see if the problem still remains. different from individual setting because they'd be sharing the same synthetic database.

# TO DO'S (10/17)


- [DONE] Do experiments with Bob first, see if Alice faces the same error. 
- [DONE] What is the average error of a query?
- [DONE] What is the query with the least and most error? 
    - To figure out what things look like for each analyst and an upper/lower bound on error
- Write code to put the functions into production for multiple tests

# To Do's (10/18)

## Conclusions...
We did a sanity check by reversing the order of Alice and Bob. 

There are mechanisms that are very specific to types of queries. H-trees are good for range queries. PMW could be the case where that PMW is good for some class of queries. We've proved that it's not

## Next…

Update steps matter, update steps matter but end synthetic database is also valuable. 

- If only the update steps matter, then if we expand these query sets to very long, they should have relatively same error as doing over the entire sequence. 

- If the final synthetic database is also valuable, then there should be a noticable difference 

try to make an adaptation where once you make your experiment, for some large number of timestamps, randomly choose a query 

25/25 normal, for next 950 query, pick one on random and ask it repeatedly

we want to see the average error from queries as time goes on. we will dilute the effect of the update steps. you should know which analysts it's from. 

Alice asks queries, bob asks queries. After they've asked their 25 each. Flip a coin. If heads, alice asks her queries at random. If tails, bob asks her queries at random. we want to see the effects of answering their queries on the synthetic database. In the case where we ask incompatible workloads, we saw there was the difference. do this once for each for the four queries, loook at the average error. 